# AI Equity Research Report Generator

In [ ]:
!pip install -q google-generativeai yfinance PyMuPDF python-dotenv

In [ ]:
import google.generativeai as genai
import os
from IPython.display import Markdown

# Load API key from Colab secrets or a local .env file.
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
except (ImportError, KeyError):
    from dotenv import load_dotenv
    load_dotenv()
    GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

if not GOOGLE_API_KEY:
    raise ValueError("Google API Key not found. Please set it up in Colab Secrets or a local .env file.")

genai.configure(api_key=GOOGLE_API_KEY)

# Configure the model
model = genai.GenerativeModel('gemini-1.5-pro-latest')
print("Gemini model configured.")

### Upload Annual Report
Run the next cell and upload the latest annual report PDF for the company you wish to analyze.

In [ ]:
from google.colab import files
import fitz  # PyMuPDF
import io

MAX_PAGES_TO_READ = 50
annual_report_text = ""

print(f"Please upload the Annual Report PDF (will read first {MAX_PAGES_TO_READ} pages)...")
uploaded_files = files.upload()

if uploaded_files:
    file_name = next(iter(uploaded_files))
    pdf_content = uploaded_files[file_name]

    with fitz.open(stream=pdf_content, filetype="pdf") as doc:
        num_pages = min(len(doc), MAX_PAGES_TO_READ)
        for page_num in range(num_pages):
            page = doc.load_page(page_num)
            annual_report_text += page.get_text()
    
    print(f"\nExtracted {len(annual_report_text):,} characters from the first {num_pages} pages of '{file_name}'.")
else:
    print("\nNo file uploaded. Report will be less detailed.")

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import date, timedelta

# --- Define Stock --- 
company_ticker = "RELIANCE"
company_name = "Reliance Industries Limited"
yf_ticker = f"{company_ticker}.NS"

# --- Fetch Live Market Data ---
print(f"Fetching live market data for {company_name}...")
market_data_context = ""
try:
    ticker = yf.Ticker(yf_ticker)
    hist = ticker.history(period="1y")
    
    if hist.empty:
        raise ValueError(f"No price data found for {yf_ticker}.")

    high_52_week = hist['High'].max()
    low_52_week = hist['Low'].min()
    last_close = hist['Close'].iloc[-1]
    yearly_return = ((hist['Close'].iloc[-1] / hist['Close'].iloc[0]) - 1) * 100
    
    news_headlines = [item.get('title') for item in ticker.news[:5] if item.get('title')]
    news_string = "\n- ".join(news_headlines)

    market_data_context = f"""
    Recent News Headlines:
    - {news_string}

    Key Stock Price Metrics:
    - Last Closing Price: ₹{last_close:,.2f}
    - 52-Week High: ₹{high_52_week:,.2f}
    - 52-Week Low: ₹{low_52_week:,.2f}
    - 1-Year Return: {yearly_return:.2f}%
    """
    print("Live market data fetched successfully.")
except Exception as e:
    print(f"Could not fetch market data from yfinance. Error: {e}")
    market_data_context = "Live market data could not be fetched."

In [ ]:
comprehensive_prompt = f"""
Act as a senior equity research analyst. Write a highly detailed equity research report for {company_name} ({company_ticker}).

Use the following two sources: the company's Annual Report text and live market data.
Synthesize information from BOTH sources to construct your analysis. Be thorough and provide insightful analysis, not just a summary.

---BEGIN ANNUAL REPORT TEXT---
{annual_report_text}
---END ANNUAL REPORT TEXT---

---BEGIN LIVE MARKET DATA---
{market_data_context}
---END LIVE MARKET DATA---

**Equity Research Report: {company_name} ({company_ticker})**

**1. Executive Summary:**
- Provide a concise overview of the business, its strategic direction, key financial highlights, and your overall investment thesis.

**2. Company Profile & Business Segments:**
- Corporate History & Vision: Summarize the company's evolution and strategic vision from the report.
- Business Segment Deep Dive: For each major business segment, analyze its performance, key drivers, and outlook based on the annual report.

**3. Industry Analysis & Competitive Landscape:**
- Analyze the industry structure for the company's key segments, discussing market size, growth drivers, and major competitors.

**4. In-Depth Financial Analysis (Based on Annual Report):**
- Income Statement Analysis: Analyze trends in revenue, EBITDA, and Profit After Tax.
- Balance Sheet Analysis: Examine debt levels (Debt-to-Equity ratio), liquidity (Current Ratio), and asset base.
- Cash Flow Analysis: Discuss cash from operations, investing, and financing activities.

**5. Management Discussion & Corporate Governance:**
- Summarize key points from the "Management Discussion and Analysis" section.
- Briefly comment on the board structure and key management.

**6. Stock Price Performance & Valuation:**
- Analyze the stock's performance using the provided live metrics.
- Discuss common valuation methodologies (e.g., P/E, EV/EBITDA) and compare to competitors if possible.

**7. Investment Risks:**
- Detail the key risks highlighted in the annual report (market, operational, regulatory).

**8. Final Investment Thesis & Recommendation:**
- Bull Case: Primary reasons for an optimistic outlook.
- Bear Case: Primary risks and reasons for caution.
- Conclusion: Conclude with a final, well-reasoned investment outlook.

**9. Disclaimer:**
- Include a standard professional disclaimer.
"""

In [ ]:
print(f"Generating comprehensive equity research report for {company_ticker}...")

try:
    response = model.generate_content(comprehensive_prompt)
    display(Markdown(response.text))
except Exception as e:
    print(f"An error occurred during generation: {e}")